In [7]:
import numpy as np
from qiskit import QuantumCircuit, Aer, execute
from qiskit.quantum_info import Operator
from qiskit.extensions import RXGate, RYGate, RZGate

In [8]:
def crear_circuito(n, tipo):
    I_f = np.array([[1, 0],
                  [0, 1]]) 
    I = np.array([[1, 0],
                  [0, 1]])
    X_f = np.array([[0, 1],
                  [1, 0]]) 
    X = np.array([[0, 1],
                  [1, 0]])    
    for q in range(n-1):
        I_f = np.kron(I_f, I)
        X_f = np.kron(X_f, X)
    J = Operator(1 / np.sqrt(2) * (I_f + 1j * X_f))    
    J_dg = J.adjoint()
    circ = QuantumCircuit(n,n)
    circ.append(J, range(n))
    if n==1:
        dx = np.pi
        dy = 0
        dz = 0
    elif n==2:    
        # Pareto sí, Nash no y Pura
        #dx = np.pi/2
        #dy = np.pi/4
        #dz = 0      

        # Pareto sí, Nash sí y Mixta
        #dx = np.pi/2
        #dy = np.pi/4
        #dz = np.random.choice([0, np.pi/2, np.pi, 3*np.pi/2])

        # Barrido
        dx = tipo[0]
        dy = tipo[1]
        dz = tipo[2]

    """
    elif n==4:
        dx = np.pi/2
        dy = 3 * np.pi/8
        dz = 3 * np.pi/4    
    """

    for q in range(n):
        circ.append(RXGate(dx),[q])
        circ.append(RYGate(dy),[q])
        circ.append(RZGate(dz),[q])         
    circ.append(J_dg, range(n))
    circ.measure(range(n), range(n))  
    return circ

In [24]:
def checkear_nozero(check):
    circ = crear_circuito(2, [_x,_y,_z])
    backend = Aer.get_backend('qasm_simulator')
    measurement = execute(circ, backend=backend, shots=1000).result().get_counts(circ)
    return ['00'] != list(measurement.keys())

In [25]:
p1 = []
probas = np.arange(0,1,0.1)                                                                     # probabilidades de ceder
for _p in probas:
    p1.append([_p])

angulos = np.arange(0, 2 * np.pi, np.pi/2)                                                      # rotaciones en x,y,z
for _x in angulos:
    for _y in angulos:
        for _z in angulos:
            check = [_x,_y,_z]
            if checkear_nozero(check):
                p1.append(check)
print("Cantidad de elementos del vector:", len(p1))

Cantidad de elementos del vector: 66
